# Cell 1: Environment check (GPU, CUDA, Python)
import sys, os, platform, subprocess, textwrap
print("Python:", sys.version.splitlines()[0])
print("Platform:", platform.platform())
# Check NVIDIA + CUDA availability
gpu_info = None
try:
    gpu_info = subprocess.check_output(["nvidia-smi", "-L"], stderr=subprocess.STDOUT).decode().strip()
except Exception as e:
    gpu_info = f"nvidia-smi not found or error: {e}"
print("GPU info:\n", gpu_info)

# We'll check torch after installation in next cell


In [ ]:
# Cell 2: Install packages (adjust CUDA version if needed)
# Replace cu118 with cu121 or cpu if your system uses a different CUDA version.
cuda_tag = "cu130"   # change if your driver/CUDA is different
index_url = f"https://download.pytorch.org/whl/{cuda_tag}"

# NOTE: run in the notebook; this will restart kernels if necessary
# Use --upgrade to get recent versions; remove --no-deps only if you want pip to manage deps.
%pip install --upgrade pip setuptools wheel
# Install torch + torchvision compiled for your CUDA
%pip install --upgrade --force-reinstall torch torchvision --index-url {index_url}

# Other ML / LangChain packages
%pip install --upgrade langchain sentence-transformers chromadb pypdf PyMuPDF transformers accelerate safetensors
# Optional: bitsandbytes for 8-bit models (only if you plan to use it and your CUDA/toolchain supports it)
# %pip install bitsandbytes


In [ ]:
# Cell 3: Verify PyTorch sees the GPU
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device name:", torch.cuda.get_device_name(0))
    print("Current device capability (major.minor):", torch.cuda.get_device_capability(0))
else:
    print("CUDA not available - check drivers/CUDA toolkit and the wheel you installed.")


In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

hf_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
print("Embeddings object created for:", hf_model_name)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = r"./healthyheart.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print("Loaded:", len(documents))


In [ ]:
import os
print(os.listdir("."))


In [ ]:
# Cell 5: Load a single PDF
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_path = r"./healthyheart.pdf"   # PDF in the same folder as notebook
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
docs = text_splitter.split_documents(documents)
print(f"Created {len(docs)} text chunks.")


In [ ]:
# Cell 6: Create embeddings for chunks (this will use GPU via torch if available)
# Create a local directory to persist Chroma
persist_dir = "./chroma_db"   # change as desired
os.makedirs(persist_dir, exist_ok=True)

# Compute embeddings and store in Chroma
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_dir)
vectordb.persist()
print("Chroma vectorstore created and persisted at", persist_dir)

# create retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 4})


In [ ]:
%pip install -U langchain-classic

In [ ]:
# Cell 7 – RetrievalQA using local Hugging Face model (NO OpenAI / GPT)

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_classic.chains import RetrievalQA

# 1) Choose a HF model (you can change this later if you want)
model_name = "google/flan-t5-base"   # safe size for RTX 3050

# 2) Load tokenizer & model (GPU if available)
device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    device=device,
)

# 3) Wrap the HF pipeline as a LangChain LLM
llm = HuggingFacePipeline(pipeline=hf_pipe)

# 4) Build the RetrievalQA chain (retriever must be created in previous cells)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

# 5) Ask a question over your heart PDF
query = "tell the perfect time table for my healty heart."
answer = qa.run(query)

print(answer)


In [ ]:
# Simple interactive Q&A loop using an existing `qa` chain

print("✅ Q&A ready! Ask anything about your PDF.")
print("Type 'exit' or 'quit' to stop.\n")

while True:
    question = input("🧠 Your question: ").strip()

    if question.lower() in {"exit", "quit", "q"}:
        print("👋 Exiting Q&A. Bye!")
        break

    if not question:
        print("Please type a question, or 'exit' to quit.\n")
        continue

    print("\nThinking...\n")
    try:
        answer = qa.run(question)   # `qa` must already be defined in previous cells
        print("💬 Answer:\n")
        print(answer)
        print("\n" + "-" * 60 + "\n")
    except Exception as e:
        print("⚠️ Error while answering:", e)
        break


In [ ]:
# Build the Hugging Face LLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_classic.chains import RetrievalQA
import torch

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = 0 if torch.cuda.is_available() else -1

hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2,
    device=device,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 🔥 Create the QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

print("QA chain is ready!")


In [ ]:
import gradio as gr

def answer_question(question):
    if not question.strip():
        return "Please enter a question."
    return qa.run(question)   # 🔥 uses your QA chain

with gr.Blocks() as demo:
    gr.Markdown("# 🫀 PDF Question Answering System")

    inp = gr.Textbox(label="Ask a Question")
    out = gr.Textbox(label="Answer", lines=10)

    btn = gr.Button("Get Answer")
    btn.click(answer_question, inputs=inp, outputs=out)

demo.launch(share=False)


In [ ]:
import gradio as gr

def respond(message, history):
    """
    message: latest user question (string)
    history: list of [user, bot] pairs (we don't even have to use it for now)
    """
    if not message.strip():
        return "Please enter a question."

    # RAG answer from your QA chain (stateless per question)
    answer = qa.run(message)
    return answer

demo = gr.ChatInterface(
    fn=respond,
    title="🫀 Heart PDF Chatbot",
    description="Ask any question about your heart health PDF.",
    examples=[
        "Give me a summary of this document.",
        "What lifestyle changes does this PDF recommend?",
        "Explain the risk factors for heart disease mentioned here."
    ]
)
def respond(message, history):
    if not message.strip():
        return "Please enter a question."

    # Build a context string from history
    convo = ""
    for user_msg, bot_msg in history:
        convo += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    convo += f"User: {message}\nAssistant:"

    # Now send the convo as the question to RAG
    answer = qa.run(convo)
    return answer


demo.launch()


In [ ]:
import gradio as gr

def respond(message, history):
    if not message.strip():
        return "Please enter a question."
    answer = qa.run(message)
    history.append((message, answer))
    return "", history


# 🔥 Inject custom CSS using HTML (Gradio 4.x compatible)
custom_css = """
<style>
body {
    font-family: 'Segoe UI', sans-serif;
    background: radial-gradient(circle at top, #1e293b 0, #020617 45%, #000000 100%);
    color: #e5e7eb;
}

/* Main container styling */
#chat-container {
    max-width: 900px;
    margin: 3rem auto;
    padding: 25px;
    border-radius: 22px;
    background: rgba(15, 23, 42, 0.85);
    box-shadow: 0 25px 60px rgba(0, 0, 0, 0.65);
    border: 1px solid rgba(148, 163, 184, 0.4);
    backdrop-filter: blur(18px);
}

/* Title */
h1 {
    font-size: 2.2rem !important;
    font-weight: 700;
    text-align: center;
    background: linear-gradient(to right, #22c55e, #38bdf8);
    -webkit-background-clip: text;
    color: transparent !important;
}

/* Chat bubbles */
.message.user {
    background: linear-gradient(135deg, #22c55e, #16a34a);
    color: white !important;
    border-radius: 16px !important;
}

.message.bot {
    background: rgba(30, 41, 59, 0.8) !important;
    border: 1px solid rgba(148, 163, 184, 0.4);
    border-radius: 16px !important;
}

/* Input box */
textarea {
    background: rgba(15, 23, 42, 0.85) !important;
    border-radius: 14px !important;
    border: 1px solid rgba(148,163,184,0.6) !important;
    color: #e5e7eb !important;
}

/* Button */
button {
    border-radius: 999px !important;
    font-weight: 600 !important;
    background: linear-gradient(135deg, #22c55e, #0ea5e9) !important;
    color: white !important;
}
</style>
"""

with gr.Blocks() as demo:

    # Add the custom CSS
    gr.HTML(custom_css)

    gr.HTML("<div id='chat-container'>")

    gr.Markdown("# 🫀 Heart Health PDF Chatbot")
    gr.Markdown("Ask any question about your PDF!")

    chatbot = gr.Chatbot(height=450)
    msg = gr.Textbox(placeholder="Ask your question here...")
    submit = gr.Button("Send")

    submit.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

    gr.HTML("</div>")

demo.launch()
